# Data Merging

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

RuntimeError: The current Numpy installation ('c:\\users\\guli\\.venvs\\lewagon_project\\lib\\site-packages\\numpy\\__init__.py') fails to pass a sanity check due to a bug in the windows runtime. See this issue for more information: https://tinyurl.com/y3dm3h86

In [2]:
%load_ext autoreload

%autoreload 2

## Fetch data from file path

Get the csv file names from a given directory

In [3]:
def get_file_names(directory):
    """returns the csv files in the given string directory path"""
    file_names = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_names.append(os.path.join(directory, filename))
    return file_names

Create a dictionary from the files where the keys are the yearly intervals and the values are the respective dataframes.

In [4]:
def get_dataframes(file_names):
    """Takes in a list of csv file paths.
    Returns a dictionary whose keys are the years
    and values are the corresponding dataframes."""
    df_years = {}
    for file in file_names:
        df_years[file[-29:]] = pd.read_csv(file) # [-29:] Indexes the years.csv eg '201501010000-201601010000.csv'
    return df_years
    

Can be used as follows:

In [5]:
directory = r'../raw_data/load/'
file_names = get_file_names(directory)
df_years = get_dataframes(file_names)

print(df_years.keys())

dict_keys(['201501010000-201601010000.csv', '201601010000-201701010000.csv', '201701010000-201801010000.csv', '201801010000-201901010000.csv', '201901010000-202001010000.csv', '202001010000-202101010000.csv'])


In [6]:
df_years['201501010000-201601010000.csv'].head(3)

,Time (CET),Day-ahead Total Load Forecast [MW] - BZN|DK1,Actual Total Load [MW] - BZN|DK1
0,01.01.2015 00:00 - 01.01.2015 01:00,1870.0,1877.0
1,01.01.2015 01:00 - 01.01.2015 02:00,1841.0,1843.0
2,01.01.2015 02:00 - 01.01.2015 03:00,1785.0,1795.0


Function to get the path per feature e.g price, load

In [7]:
def get_feature_path(feature_name, main_directory):
    for root, dirs_, files in os.walk(main_directory):
        if feature_name in (root):
            return root

Function to bring it together and return a tuple containing a nested dictionary of the features and the corresponding dictionary of per year dataframes. 

In [8]:
def get_features_df(main_path):
    feats = ['price', 'load']
    feat_dict = {}
    for feat in feats:
        path = get_feature_path(feat, main_path)
        names = get_file_names(path)
        feat_dict[feat] = get_dataframes(names)
        
    return feat_dict[feats[0]], feat_dict[feats[1]]

Used as follows...

In [9]:
path = r'../raw_data/'
price, load = get_features_df(main_path=path) # unpack tuple 

## Create DataFrames from data

In [10]:
test_df = price['202001010000-202101010000.csv']

In [11]:
test_df[test_df['MTU (CET)'] == '25.11.2020 00:00 - 25.11.2020 01:00']

,MTU (CET),Day-ahead Price [EUR/MWh]
7897,25.11.2020 00:00 - 25.11.2020 01:00,-


### Concatenate dataframes

In [12]:
def concat_dataframes(feat):
    keys = list(feat.keys())
    df = pd.concat([
        feat[keys[0]],  # year 2015
        feat[keys[1]],  # year 2016
        feat[keys[2]],  # year 2017
        feat[keys[3]],  # year 2018
        feat[keys[4]],  # year 2019
        feat[keys[5]]   # year 2020
    ]).reset_index(drop=True)
    return df

In [13]:
prices_data = concat_dataframes(price)
load_data = concat_dataframes(load)

In [14]:
prices_data[prices_data['MTU (CET)'] == '24.11.2020 00:00 - 24.11.2020 01:00']

,MTU (CET),Day-ahead Price [EUR/MWh]
51702,24.11.2020 00:00 - 24.11.2020 01:00,0.11


In [15]:
prices_data.isnull().sum()

MTU (CET)                    0
Day-ahead Price [EUR/MWh]    6
dtype: int64

### Convert to datetime

On peeking through the dataframes nested in the dictionaries `price` and `load`, the columns showing the time display them as time ranges e.g `31.12.2020 19:00 - 31.12.2020 20:00`. A function is created to make a new column called _time_ that will strip the string down to the initial timestamp e.g `31.12.2020 19:00` and then convert the series to datetime objects

In [16]:
def get_datetime(df):
    try:
        column = df['MTU (CET)'] # column with time values in price df
    except:
        column = df['Time (CET)']  # load time
    
    # create new column 'time' by formatting the original time column to get single timepoint instead of a range
    df['time'] = column.apply(lambda _: _[:16])
    # convert new time column from str to timestamp
    df['time'] = pd.to_datetime(df['time'])
        
    return df

In [17]:
load_data = get_datetime(load_data)
prices_data = get_datetime(prices_data)

In [18]:
prices_data['time']

0       2015-01-01 00:00:00
1       2015-01-01 01:00:00
2       2015-01-01 02:00:00
3       2015-01-01 03:00:00
4       2015-01-01 04:00:00
                ...        
52609   2020-12-31 19:00:00
52610   2020-12-31 20:00:00
52611   2020-12-31 21:00:00
52612   2020-12-31 22:00:00
52613   2020-12-31 23:00:00
Name: time, Length: 52614, dtype: datetime64[ns]

### Merge datasets

In [19]:
print(len(prices_data))
print(len(load_data))

52614
52614


In [20]:
df = prices_data.merge(load_data, on='time').reset_index(drop=True)  # merge both

In [35]:
def merge_data(feat_1, feat_2):
    df = feat_1.merge(feat_2, on='time').reset_index(drop=True)
    return df

### Select valid time range

In [21]:
idx_p = prices_data[prices_data['time'] == '2020-11-23 23:00:00'].index  # date up until
idx_l = load_data[load_data['time'] == '2020-11-23 23:00:00'].index

assert(idx_p == idx_l) # to check if same index on both dataframes

idx = idx_l[0] + 1  # to slice from dataframe

del idx_l
del idx_p

In [22]:
df = df.iloc[:idx]

### Drop unnecessary columns

In [23]:
to_drop = ['MTU (CET)', 'Time (CET)', 'Day-ahead Total Load Forecast [MW] - BZN|DK1']
df.drop(columns=to_drop, inplace=True)

In [24]:
df.columns

Index(['Day-ahead Price [EUR/MWh]', 'time',
       'Actual Total Load [MW] - BZN|DK1'],
      dtype='object')

### Handle missing data

In [25]:
df.isnull().sum()

Day-ahead Price [EUR/MWh]           6
time                                0
Actual Total Load [MW] - BZN|DK1    8
dtype: int64

In [26]:
df['Day-ahead Price [EUR/MWh]'][df['Day-ahead Price [EUR/MWh]'].isnull() == True]

2090     NaN
10829    NaN
19568    NaN
28307    NaN
37214    NaN
45953    NaN
Name: Day-ahead Price [EUR/MWh], dtype: object

In [27]:
df['Actual Total Load [MW] - BZN|DK1'][df['Actual Total Load [MW] - BZN|DK1'].isnull() == True]

2090     NaN
10829    NaN
16038    NaN
16040    NaN
19568    NaN
28307    NaN
37214    NaN
45953    NaN
Name: Actual Total Load [MW] - BZN|DK1, dtype: object

In [28]:
imputer = SimpleImputer()

df[['Day-ahead Price [EUR/MWh]','Actual Total Load [MW] - BZN|DK1']] = imputer.fit_transform(df[['Day-ahead Price [EUR/MWh]',
                                                                                                 'Actual Total Load [MW] - BZN|DK1']])

In [29]:
df.isnull().sum()

Day-ahead Price [EUR/MWh]           0
time                                0
Actual Total Load [MW] - BZN|DK1    0
dtype: int64

In [30]:
# total_df['hour'] = total_df['time'].apply(lambda x: x.hour)
# total_df = total_df[total_df['hour']==0].reset_index(drop=True)  # index by 

In [31]:
df.head()

,Day-ahead Price [EUR/MWh],time,Actual Total Load [MW] - BZN|DK1
0,25.02,2015-01-01 00:00:00,1877.0
1,18.29,2015-01-01 01:00:00,1843.0
2,16.04,2015-01-01 02:00:00,1795.0
3,14.60,2015-01-01 03:00:00,1745.0
4,14.95,2015-01-01 04:00:00,1743.0


## Final function

In [38]:
def fetch_data(path):
    
    price, load = get_features_df(main_path=path) # unpack tuple
    
    prices_data = concat_dataframes(price)
    load_data = concat_dataframes(load)

    load_data = get_datetime(load_data)
    prices_data = get_datetime(prices_data)

    # date up until
    idx_p = prices_data[prices_data['time'] == '2020-11-23 23:00:00'].index
    idx_l = load_data[load_data['time'] == '2020-11-23 23:00:00'].index
    assert(idx_p == idx_l)
    idx = idx_l[0] + 1
    
    df = merge_data(prices_data, load_data)
    df = df.iloc[:idx]

    # drop unused columns
    to_drop = ['MTU (CET)', 'Time (CET)', 'Day-ahead Total Load Forecast [MW] - BZN|DK1']
    df.drop(columns=to_drop, inplace=True)

    # handle missing data
    imputer = SimpleImputer()

    df[['Day-ahead Price [EUR/MWh]','Actual Total Load [MW] - BZN|DK1']] \
        = imputer.fit_transform(df[['Day-ahead Price [EUR/MWh]','Actual Total Load [MW] - BZN|DK1']])

    return df

### Test

In [40]:
df = fetch_data(path)

,Day-ahead Price [EUR/MWh],time,Actual Total Load [MW] - BZN|DK1
0,25.02,2015-01-01 00:00:00,1877.0
1,18.29,2015-01-01 01:00:00,1843.0
2,16.04,2015-01-01 02:00:00,1795.0
3,14.60,2015-01-01 03:00:00,1745.0
4,14.95,2015-01-01 04:00:00,1743.0
...,...,...,...
51697,4.08,2020-11-23 07:00:00,3182.0
51698,6.99,2020-11-23 08:00:00,3255.0
51699,6.95,2020-11-23 09:00:00,3213.0
51700,5.92,2020-11-23 10:00:00,3181.0


In [32]:

    # return merged and cleaned dataframe   
    
    # get path per feature
    #get_feature_path(feat, main_path)
    
    # get filenames per feature per path

    # get dataframes per feature

    # concatenate dataframes per feature dropping duplicate times

    # merge features on time index or column

    # return merged dataframe